In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding = "ISO-8859-1")
df.dropna(axis=0,inplace=True)
#df

In [ ]:
Y=df.iloc[:,5]
Y=pd.get_dummies(Y)
#Y

In [ ]:
X=df['OriginalTweet']
X=np.array(X)
#X

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
lem=WordNetLemmatizer()

In [ ]:


for i,sent in enumerate(X):
  #print(i)
  #sent=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sent)

  sent=sent.lower()
  sent=re.sub('https:\/\/\S+','',sent)
  sent=re.sub('@\S+','',sent)
  sent=re.sub('&[a-zA-Z]+;','',sent)
  sent=re.sub('#\S+','',sent)
  sent=re.sub('[^a-zA-Z0-9]',' ',sent)
  sent=sent.split()
  wds=''
  for word in sent:
    #if word not in stopwords.words('english'):
      word=lem.lemmatize(word)
      wds+=' '
      wds+=word
  X[i]=wds
#X

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=100000)
tokenizer.fit_on_texts(X)
dic=tokenizer.word_index
#print(dic)


In [ ]:
X_train=tokenizer.texts_to_sequences(X)
#X_train

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,maxlen=100,padding='pre')
#X_train

In [ ]:
import os
fl=open('../input/glove6b/glove.6B.300d.txt',encoding='utf-8')
GlvDic={}
for line in fl:
  lst=line.split()
  wrd=lst[0]
  ary=np.array(lst[1:])
  GlvDic[wrd]=ary
  #print(GlvDic[wrd])


In [ ]:
mat=np.zeros((100000,300))
for key,i in dic.items():
  vec=GlvDic.get(key)
  if vec is not None:
    mat[i]=vec
    


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional

In [ ]:
model=Sequential()
model.add(Embedding(100000,300,input_length=300))
model.add(Bidirectional(LSTM(500)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5,activation='softmax'))
model.layers[0].set_weights([mat])
#model.layers[0].trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y,validation_split=.2,epochs=20,batch_size=15)